# Lecture 3.11: Neural Networks Pt.1

[**Lecture Slides**](https://docs.google.com/presentation/d/1ZXTfGXZmzQfeUQcGkCR7ejav-29ekTyjwU0pFq3Vi-8/edit?usp=sharing)

This lecture, we are going to load neural network weights into keras models, and apply them to regression and classification tasks.

**Learning goals:**
- distinguish keras and tensorflow libraries
- discover the keras Sequential and Model api
- load HDF5 model weights
- predict and visualize a neural network classifier
- predict and visualize a neural network regressor
- understand the shapes of layers and weight matrices


## 1. Setup

As our machine learning adventure continues, we wish to build neural networks, and must leave our trusted companion, sklearn. 😢 Instead, we will use a deep learning framework: _keras_ 🤠  

[Keras](https://keras.io/) is an _API_. It is just a friendly interface to work with neural networks, and doesn't specify the implementations. Therefore, keras must always be used with a _backend_ to do the mathematical heavy lifting (although this will change [soon](https://datascience.stackexchange.com/questions/65736/why-does-keras-need-tensorflow-as-backend)). We will use [tensorflow](https://www.tensorflow.org/) as our keras backend.

This means we have to install two new python dependencies: `keras` and `tensorflow`. Just like all the other packages we've used so far, these were already added to the `Pipfile` included in this repo. However, your `pipenv install` might fail during the locking stage, as the last stable pipenv release is [incompatible](https://github.com/pypa/pipenv/issues/3921) with tensorflow 2.0. To fix this, we must install the latest _beta_ release of pipenv.

We can do this through the `pip` installed on our `pyenv` python interpreter. If these terms are confusing, please check out lecture 1.1 or this [blogpost](https://hackernoon.com/reaching-python-development-nirvana-bb5692adf30c). 

First, make sure you are **not** inside your pipenv shell. Then, upgrade pipenv to the latest beta release with:

```bash
pip install -U pipenv==2020.4.1b1
```

_Note: If `pip` complains at this stage, you might have to install the `wheel` package, which you can do through `pip install wheel`._

You should now be able to install the `Pipfile` libraries as per usual, by running `pipenv install` in the repo's root directory.

_Note: Tensorflow is huge and has many dependencies. This means the locking stage can take a while, up to ~ 15 minutes. You can always skip locking with `pipenv install --skip-lock` if you are impatient_ ⌛️


## 2. Classification

### 2.1 Data Munging

Let's see if neural networks can help use detect fake banknotes. We'll use the same [banknote authentication dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication) as the last lectures , and try to solve the binary classification task.

In [ ]:
import pandas as pd

df = pd.read_csv('bank_note.csv')

df.head()

We choose `feature_2` and `feature_4` as features, which we can visualize:

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5,5), dpi=120)
ax = fig.add_subplot()

scatter = ax.scatter(df.iloc[:, 1], df.iloc[:, 3], c=df.iloc[:, 4], cmap=plt.cm.coolwarm, s=20, edgecolors='k', alpha=0.5)
ax.set_xlabel('feature_2')
ax.set_ylabel('feature_4')
ax.set_title('Banknote Classification')
handles, labels = scatter.legend_elements()
ax.legend(handles=handles, labels=['genuine', 'fake']);

Recall that data is _not separable_ , and the relationship between `feature_2` and `feature_4` is _non-linear_. This should make a good challenge for our neural networks!

The features are standardized, so we can directly create our feature matrix, `X`, and our label vector, `y`:

In [ ]:
X = df[['feature_2', 'feature_4']].values
y = df['is_fake'].values

### 2.2 Loading Weights

We haven't learned about neural network _training_ yet (spoiler alert ⚡️: we'll be covering optimization next lecture), so instead of _fitting_ a model to our dataset, we'll _load_ a previously trained model. 💾

The model weights for our banknote classifier are stored as a HDF5 file: `banknote_v1.h5`. Before we can load them however, we need to build the _architecture_ of our neural network. 🏗 This sets up an empty node structure, into which we can fill the saved $\Theta^{(l)}$ values. 

Of course, the neural network structure needs to match that of the saved weights. For this model, we'll be loading a neural network with _one hidden layer of 10 neurons_.

We'll use the keras [Sequential api](https://keras.io/guides/sequential_model/) to create our neural network structure. This makes it easy to stack layers on top of each other. 

As we'll discover in the next few lectures, there are many types of neural network layers. The simple layers we saw in the lecture slides are called _dense layers_. These can be instantiated with the keras [Dense](https://keras.io/api/layers/core_layers/dense/) class. These can be chained in a list and passed as argument to our `Sequential` constructor.

⚠️ Keras uses a slightly different terminology than the lecture slides. We have defined the input features to form the first layer of the neural network. Keras considers the network to start with the first _hidden_ layer. This can get confusing, so we often describe a neural network architecture by its number of hidden layers, which everyone agrees on.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras import Input

model = Sequential([
    Dense(units=10, activation='sigmoid', input_dim=2), 
    Dense(units=1, activation='sigmoid')
])

We have built a (empty) neural network! 🎊

A few important arguments:
- `units` is the number of neurons in that layer. Note how the hidden layer has more units than the number of input features: this allows the model to calculate complex artificial features. The output later has one unit since we are dealing with a _binary_ classification task. We therefore expect _one_ class probability as output.
- `activation` is our layer's activation function. This will be applied to all the neurons in that layer. The hidden layers _must_ have a non-linear activation function. `sigmoid` is just a synonym of logistic, so we are using a logistic function activation here. The last layer also uses `sigmoid` since we are tackling a _classification task_. We therefore want the last layer to act as a logistic regression model.
- `input_dim` defines the shape of the _feature vectors_. In our case,  we have 2 features, so 2D vectors.

Notice how we only have to define the shape of our _input_. Keras figures out how to connect all the other layers from their _units_. This includes the shapes of the weight matrices. i.e keras transforms the _structural_ information we supplied as arguments into the chained _mathematical_ functions we described in the lecture slides. This prevents us from doing linear algebra everytime we want to train a neural network! 😅

ℹ️ Vectors, matrices, and tensors in neural networks are notoriously confusing, so it's still worth trying to keep track of the input/output shapes as they pass through layers. This [stackoverflow post](https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc) is a great summary.



The `Sequential` api is fairly easy to read, but neural networks can get very complex and we might want to _visualize_ their architecture. Keras helps us out by providing a printable `.summary()` method:

In [ ]:
print(model.summary())

We find the 2 keras layers and their respective units: 10 & 1. But are also shown the number of model _weights_ that were setup for us in the background. Let's check if keras has paid attention during linear algebra class, and correctly inferred the number of parameters $\theta$ 👩‍🏫

Our input feature vectors have two dimensions:

$$ 
\textbf{x} = \begin{bmatrix}x_{1}\\x_{2}\end{bmatrix}; 
$$

Therefore our first hidden layer activation has 2 model parameters, plus 1 bias term $\theta_{0}$:

$$
a_{j}^{(2)} = sigmoid(\theta_{j0}^{(1)} + \theta_{j1}^{(1)} x_{1} + \theta_{j2}^{(1)} x_{2})
$$

So 3 $\theta$ values per activation. Since we have 10 hidden neurons, we expect $3 \times 10 = 30$ weights mapping our input to the hidden layer.

10 neurons in the hidden layer means 10 activations fed into the output layer. With an added bias term, this makes 11 $\theta$s.

$$
a^{(3)} = sigmoid(\theta_{0}^{(2)} + \theta_{1}^{(2)} a_{1}^{(2)} + \theta_{2}^{(2)} a_{2}^{(2)} + \theta_{3}^{(2)} a_{3}^{(2)} + \theta_{4}^{(2)} a_{4}^{(2)} + \theta_{5}^{(2)} a_{5}^{(2)} + \theta_{6}^{(2)} a_{6}^{(2)} + \theta_{7}^{(2)} a_{7}^{(2)} +  \theta_{8}^{(2)} a_{8}^{(2)} +  \theta_{9}^{(2)} a_{9}^{(2)} +  \theta_{10}^{(2)} a_{10}^{(2)})
$$

(Can you imagine having to type this everytime we are creating a model... 🙈)  

Since there is only one output neuron, there there is only one activation $a^{(3)}$, and we expect $11$ weights mapping the hidden layer to the output later. We have $30+11=41$ total model parameters, no wonder neural networks are powerful! It looks like keras is pretty good at maths. 🤓

🧠 Please take your time to understand the above, and how one layer maps to the next. Check this lecture's slides for a visual explanation.

ℹ️ The layer shapes have a strange `None` as first dimension. This refers to the _batch size_ and we'll see it in detail in the next couple of lectures.

We can also use keras' [`.plot_model()`](https://keras.io/api/utils/model_plotting_utils/) to see the layer structure. This uses a library called graphviz which must installed outside of python, so we'll load the `.png` directly from this repo.

If you want to generate the image it on your own, install [graphviz](https://www.graphviz.org/), then run:

``` python
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='banknote_model_plot.png', show_shapes=True, show_layer_names=False, dpi=200, rankdir='LR')
```

Otherwise, here it is:

In [ ]:
from IPython.display import Image
Image(filename='banknote_model_plot.png') 

Once again, we see our layers and their mappings. This might not be the most useful with only one hidden layer, but could help out when using 50!

Now that we have our neural network architecture ready, let's populate its weights. We can use keras' [`.load_weights()`](https://keras.io/api/models/model_saving_apis/#loadweights-method), as these were stored in a HDF5 format:

In [ ]:
model.load_weights('banknote_v1.h5')

If something goes wrong at this step (and it often will), there must be a discrepancy between our neural structure and the model we are loading. In our case, everything seems to have worked. 😌

### 2.3 Prediction

Now that we have loaded our neural network, let's use it to predict the authencity of some banknotes! We'll use the same tricky example as the two previous lectures: the `geniune` cluster on the left hand of the dataset plot, near $[-1, 0]$.

The keras [model api](https://keras.io/api/models/model_training_apis/) is close to the sklearn api, so it'll feel familiar when we use the `.predict()` (and later `.fit()`) methods. Let's reshape our single data point into a $1 \times 2$ matrix, and predict its class:

In [ ]:
import numpy as np

x_predict = np.array([-1, 0]).reshape(1, 2)
y_predict = model.predict(x_predict)
y_predict

... class of 0.05? One key difference between the sklearn and keras `.predict()`, is that the latter returns the _probability_ of the logistic output node in the binary classification case. This is a great example of why one must always read the documentation closely!

This means that our example $[-1, 0]$ has a 5% chance of being fake 

🧠 How do we know that it's 5% chance of being `fake` and not the other way round, a 5% chance of being genuine?

We can turn this into a prediction by using a simple 0.5 threshold:

In [ ]:
y_predict > 0.5

Our tricky example was correctly classified by our neural network! 🤖

### 2.4 Analysis

We want to have an overview of _all_ the model's predictions. Since this classification task only deals with two input features, we can visualize the neural network's _decision boundary_.

This time, we put all the visualization helper functions from last lecture in their own _module_ , `plot.py`. The file was placed in the same directory as this notebook, so we can load it with `from plot import plot_classification`:

In [ ]:
from plot import plot_classification

fig = plt.figure(figsize=(5,5), dpi=120)
ax = fig.add_subplot()
plot_classification(ax, X, y, model)

😮 That's a cool looking model! Notice how the neural network can accurately predict the mixed clusters with its non-linear decision boundary. Recall that this is possible thanks to its artificial features, which theoretically allow it to fit _any_ mapping from features to labels. The output layer is a simple logistic regression model, operating on these higher order features.

In this case, the network looks accurate: enough power to capture the dataset's clustered topology, but not too much to overfit to single data points. 

Let's bring in our buddies, SVM & random forest, to compare against this neural network (refer to lectures 3.9 & 3.10 for a refresher):


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from plot import compare_classification

svm_rbf = SVC(kernel='rbf', random_state=0)
svm_rbf = svm_rbf.fit(X, y)

forest_clf = RandomForestClassifier(random_state=0)
forest_clf = forest_clf.fit(X, y)

compare_classification(X, y, [svm_rbf, forest_clf, model], ['RBF SVM', 'Random Forest', 'Neural Network'])

Compared to SVMs and Random Forests, it's harder to _interpret_ the decision boundary of neural networks. This is because they create their own feature space... so only they knows why they make a particular prediction!

This is still a good overview of the strengths and weaknesses of each model's predictive power: notice how the shapes are very different even for the same binary classification task on this low dimensional dataset. It's clear that there is no one model to rule them all! 💍

🧠 Why are there color gradients in the neural network graph only? What do they represent?

## 3. Regression

### 3.1 Data Munging

Let's try out neural networks on a _regression task_. We'll use the `instagram_planning` dataset from the last few lectures to test its non-linear prediction power.

In [ ]:
import pandas as pd

df = pd.read_csv('instagram_planning.csv')
df.head()

We can create our feature matrix and label vector, and visualize them:

In [ ]:
X = df['planned_minutes'].values.reshape(-1, 1)
y = df['actual_minutes'].values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X, y, alpha=0.6)

ax.set_xlabel('planned online time (min)')
ax.set_ylabel('time spent online (min) ')
ax.set_title('Online Procrastination');

It's good practice to standardize our features: neural networks optimization is sensitive to feature scales. In this case, the saved model weights were learned on a standardized version of this dataset, so we _must_ apply the same transformation here to get the correct predictions. This is a great reminder of how important good data engineering is, to keep track of all these dataset changes!

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(X)
X_scaled = scaler.transform(X)

### 3.2 Prediction

This time we'll be using a neural network with 1 hidden layer of 6 neurons. We can use the same `Sequential` api as for classification, except we'll be using a `linear` activation for our output layer. This tells keras to not use any activation function on our output, only the $\boldsymbol{\theta}.\textbf{a}$ hypothesis. 

i.e a `Dense` layer with a `linear` activation, is the _same_ as a layer of linear regressions. A `Dense` layer with a `sigmoid` activation, is the same as a layer of logistic regressions.

In [ ]:
model = Sequential([
  Dense(6, activation='sigmoid', input_dim=1),
  Dense(1, activation='linear')
])

Note that the `input_dim` is 1 this time, because we are using a single feature: `planned_minutes`

We can now load our model weights:

In [ ]:
model.load_weights('instagram_planning_model_v1.h5')

We can predict how much we'll spend online if we originally plan _20 minutes_ , by making the following feature matrix:

In [ ]:
x_predict = np.array([20]).reshape(1, 1)
x_predict_scaled = scaler.transform(x_predict)

Remember we _must always_ transform features with the same preprocessors as were used during training.

keras uses the same `.predict()` api for any neural network architecture:

In [ ]:
model.predict(x_predict_scaled)

Our neural network guessed that we'll spend about an hour online. 📱

### 3.3 Analysis

We want to have an overview of all the model's predictions. Since this regression task deals with only one input feature, we can visualize them directly:

In [ ]:
from plot import plot_regression

fig = plt.figure(figsize=(5,5), dpi=120)
ax = fig.add_subplot()
plot_regression(ax, X, y, model, scaler)

Notice how the neural network correctly models the slightly non-linear relationship between the x & y axes. It's the 6 hidden neurons that allow the final linear regression layer to act a universal function approximator.

Let's compare it to linear regression and random forest regression:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from plot import compare_regression    


lr = LinearRegression()
lr = lr.fit(X_scaled, y)

forest_reg = RandomForestRegressor(random_state=0)
forest_reg = forest_reg.fit(X_scaled, y)

compare_regression(X, y, [lr, forest_reg, model], ['Linear Regression', 'Random Forest', 'Neural Network'], scaler)

Once again, it's hard to give an intuition of _why_ the neural network predictions take this appearance, but these graphs are quite typical of each model type. Note how the NN line isn't as jagged as the random forest, but has more non-linear potential than linear regression. 

Just like for classification, there is no single best model for all tasks and datasets, and seeing these hypotheses gives an idea of how each regressor has strengths and weaknesses.

That said, it's clear that neural networks have a lot of generalization power, and this is why they're so popular 😎

## 4. Summary

Today, we learned about classification and regression with **neural networks**. First, we used parallel logistic regression models to create **artificial features**. We chained these **layers** to create powerful **nested operations**. We understood that this framework of chained non-linear operations acted as a **universal function approximator**. These are capable of modeling complex datasets and are called **neural networks**. We represented these models as a graph, and calculated predictions by stepping through its nodes. We defined important NN vocabulary, such as **neurons**, **activations**, **layers**, and **weights**. Finally, we then created empty neural network structures in **keras**, and loaded model weights to solve classification and regression tasks.


# Resources

### Core Resources

- [**Slides**](https://docs.google.com/presentation/d/1ZXTfGXZmzQfeUQcGkCR7ejav-29ekTyjwU0pFq3Vi-8/edit?usp=sharing)
- [introduction to neural networks](https://victorzhou.com/blog/intro-to-neural-networks/)  
Great visual blog post
- [building your first NN in keras](https://victorzhou.com/blog/keras-neural-network-tutorial/)  
Same blogger about first steps in keras

### Additional Resources

- [Feature visualization in NNs](https://distill.pub/2017/feature-visualization/)  
Visual summary of Olah et Al's great 2017 paper on CNN feature visualization
- [DirtyBiology - biomimetism](https://youtu.be/1qvJDQTqSHY)  
Youtube video on biomimetism and the evolutionary forces of technical progress. In French, so put the subtitles on!
- [Input/output shapes in keras](https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc)  
Stackoverflow thread summarizing the different shape / dimension arguments in the keras api
- [History of keras and tensorflow](https://datascience.stackexchange.com/questions/65736/why-does-keras-need-tensorflow-as-backend)  
Stackoverflow thread describing the complicated relationship between the frameworks, and what to expect in the future
